### Load modules

In [1]:
import pandas as pd
import numpy as np
import os, re 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")

#helper function for parsing the names later on
def parse_data(data):
    parsed_data = {}
    current_section = None
    for line in data.splitlines():
        # Check if the line starts a new section
        section_match = re.match(section_regex, line)
        if section_match:
            current_section = section_match.group().strip(':')
            parsed_data[current_section] = []
        elif current_section:  # Add billboard data to the current section
            billboards = re.findall(billboard_regex, line)
            if billboards:
                parsed_data[current_section].extend(billboards)
    return parsed_data

# Regular expressions to identify the different sections
section_regex = r"(Participant ID:\s\w+|passed_billboard_index_list:|passed_fixated_first_list:|passed_fixated_incepted_list:|repeated_forgone_list:|forgone_list:|passed_list:|glanced_list:)"
billboard_regex = r"billboard_image_\d+"

#### Setting up subjects and result data frame

In [2]:
sub_list = [f'sub{str(i).zfill(3)}' for i in range(1, 41)]
n_subs = len(sub_list)
print(sub_list[:3])

['sub001', 'sub002', 'sub003']


In [8]:
result_df = pd.DataFrame(np.zeros((40, 6)), columns = ['subj', 'condition',
                                                        'billboard_name', 'viewing_status', 
                                                        'recall_status', 'recognition_status' ])

result_df.tail(3)

,subj,condition,billboard_name,viewing_status,recall_status,recognition_status
37,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
result_df_list = []
condition_vector = []

for current_subject in  sub_list:
    #print(current_subject)
       
    #### Set up the file names to be read for this subject
    position_names_file        = '../data/01_gaze_data/' + current_subject + '_billboard_list_position_name_assignment.csv'
    passing_viewing_list_file  = '../data/01_gaze_data/' + current_subject + '_lists_1.txt'
    recall_file                = '../data/03_interview_qualtrics_data/recall_sheet.csv'
    recognition_file           = '../data/03_interview_qualtrics_data/recognition_sheet.csv'
    
    #######################################################################################
    
    #### Reading the position_name file
    #### this gives us the "raw names" of which billboards were for this ppt at which position, which will be used to relabel the output files.
    df_position_names  = pd.read_csv(position_names_file)
    df_position_names['billboard_name_at_position_cleaned'] = df_position_names['billboard_image_at_position'].apply(lambda x: x.replace("resources/billboards/", "").replace(".bmp", ""))
    df_dropped = df_position_names.drop(['index', 'position_index', 'billboard_image_at_position'], axis=1)
    mapping_dict = pd.Series(df_dropped['billboard_name_at_position_cleaned'].values, index=df_dropped['billboard_named_position']).to_dict()
    
    #######################################################################################
    
    #### Reading the file with the remembered billboards
    #### This gives us the behavioral information, i.e. which bbs were looked at, not looked at, incepted, etc.
    with open(passing_viewing_list_file, 'r', encoding='utf-8') as file:
        data = file.read()
    parsed_data = parse_data(data)
    df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in parsed_data.items()])) # Convert to a DataFrame (optional)
    
    passed_fixated_first_list = df['passed_fixated_first_list'].dropna()
    passed_fixated_first_list = list(passed_fixated_first_list.values)
    passed_fixated_first_list = [mapping_dict.get(item, item) for item in passed_fixated_first_list]
    
    passed_fixated_incepted_list = df['passed_fixated_incepted_list'].dropna()
    passed_fixated_incepted_list = list(passed_fixated_incepted_list.values)
    passed_fixated_incepted_list = [mapping_dict.get(item, item) for item in passed_fixated_incepted_list]
    
    repeated_forgone_list = df['repeated_forgone_list'].dropna()
    repeated_forgone_list = list(repeated_forgone_list.values)
    repeated_forgone_list = [mapping_dict.get(item, item) for item in repeated_forgone_list]
    
    forgone_list = df['forgone_list'].dropna()
    forgone_list = list(forgone_list.values)
    forgone_list = [mapping_dict.get(item, item) for item in forgone_list]
    
    passed_list = df['passed_list'].dropna()
    passed_list = list(passed_list.values)
    passed_list = [mapping_dict.get(item, item) for item in passed_list]
    
    glanced_list = df['glanced_list'].dropna()
    glanced_list = list(glanced_list.values)
    glanced_list = [mapping_dict.get(item, item) for item in glanced_list]
    
    # Create a list of lists and then a DataFrame from the list of lists
    all_lists = [passed_fixated_first_list, passed_fixated_incepted_list, repeated_forgone_list, forgone_list]
    result_df = pd.DataFrame( [item for sublist in all_lists for item in sublist], columns=['billboard'])
    
    result_df['viewing_status'] = ( ['passed_first_fixated'] * len(passed_fixated_first_list) +  
                                    ['passed_incept_fixated'] * len(passed_fixated_incepted_list) +  
                                    ['passed_incept_notfixated_forgone'] * len(repeated_forgone_list) + 
                                    ['passed_first_notfixated_forgone'] * len(forgone_list) )

    fixated_list = passed_fixated_first_list + passed_fixated_incepted_list
    result_df['fixated']  = [1 if item in fixated_list else 0 for item in result_df['billboard']]
    
    result_df['glanced']  = [1 if item in glanced_list else 0 for item in result_df['billboard']]

    #######################################################################################
    
    #### Reading the recall data file
    recall_df              = pd.read_csv(recall_file)
    this_sub_df            = recall_df[recall_df['subject'] == current_subject]
    this_sub_recalled      = this_sub_df.columns[this_sub_df.iloc[0] == 1].tolist()
    result_df['condition'] = this_sub_df['condition'].values[0]
    condition_vector.append(this_sub_df['condition'].values[0])
    result_df['recalled']  = [1 if item in this_sub_recalled else 0 for item in result_df['billboard']]
    
    #######################################################################################
    
    #### Reading the Recognition Data
    recognition_df          = pd.read_csv(recognition_file)
    this_sub_df             = recognition_df[recognition_df['subject'] == current_subject]
    this_sub_recognized     = this_sub_df.columns[this_sub_df.iloc[0] == 1].tolist()
    result_df['recognized'] = [1 if item in this_sub_recognized else 0 for item in result_df['billboard']]
    result_df['subject']    = current_subject

    #reorder, append, and delete
    result_df = result_df[['subject', 'condition', 'billboard', 'viewing_status', 'fixated', 'glanced', 'recalled', 'recognized']]
    result_df_list.append(result_df)
    del result_df


# Concatenate the DataFrames vertically
all_sub_df = pd.concat(result_df_list, ignore_index=True)
all_sub_df

,subject,condition,billboard,viewing_status,fixated,glanced,recalled,recognized
0,sub001,Free_Viewing,lawyer,passed_first_fixated,1,1,0,0
1,sub001,Free_Viewing,texting_driving,passed_first_fixated,1,1,1,1
2,sub001,Free_Viewing,burger,passed_first_fixated,1,1,0,0
3,sub001,Free_Viewing,binge_drinking,passed_first_fixated,1,1,0,1
4,sub001,Free_Viewing,employee_wellness,passed_first_fixated,1,1,0,1
...,...,...,...,...,...,...,...,...
970,sub040,Count_trash_bins,furniture,passed_first_notfixated_forgone,0,1,0,0
971,sub040,Count_trash_bins,great_lakes,passed_first_notfixated_forgone,0,0,0,0
972,sub040,Count_trash_bins,moving_business,passed_first_notfixated_forgone,0,0,0,0
973,sub040,Count_trash_bins,caffeine_consumption,passed_first_notfixated_forgone,0,0,0,0


In [89]:
all_sub_df.to_csv('../data/04_statistics/item_level_results.csv')


In [90]:
result = all_sub_df.groupby(['subject', 'viewing_status']).size().reset_index(name='count')
result

,subject,viewing_status,count
0,sub001,passed_first_fixated,17
1,sub001,passed_first_notfixated_forgone,7
2,sub001,passed_incept_fixated,7
3,sub002,passed_first_fixated,3
4,sub002,passed_first_notfixated_forgone,8
...,...,...,...
130,sub039,passed_incept_fixated,2
131,sub040,passed_first_fixated,3
132,sub040,passed_first_notfixated_forgone,9
133,sub040,passed_incept_fixated,5


In [92]:
pivot_table = result.pivot_table(index='subject', columns='viewing_status', values='count', fill_value=0)
pivot_table['condition'] = condition_vector
pivot_table

viewing_status,passed_first_fixated,passed_first_notfixated_forgone,passed_incept_fixated,passed_incept_notfixated_forgone,condition
subject,,,,,
sub001,17,7,7,0,Free_Viewing
sub002,3,8,5,3,Count_trash_bins
sub003,28,2,5,0,Free_Viewing
sub004,1,14,0,5,Count_trash_bins
sub005,16,7,6,0,Free_Viewing
sub006,8,11,5,1,Free_Viewing
sub007,30,3,3,0,Free_Viewing
sub008,5,15,2,2,Count_trash_bins
sub009,25,8,3,0,Free_Viewing


In [94]:
print(pivot_table['passed_first_fixated'].mean())
pivot_table['passed_first_fixated'].std()

11.2


10.785079912450751

In [97]:
pivot_table.groupby('condition').mean()

viewing_status,passed_first_fixated,passed_first_notfixated_forgone,passed_incept_fixated,passed_incept_notfixated_forgone
condition,,,,
Count_trash_bins,3.05,9.30,2.40,3.95
Free_Viewing,19.35,6.05,4.05,0.60


In [102]:
4.05/4.65




0.8709677419354838

In [78]:
pivot_table['passed_incept_fixated'].mean()

3.225

In [79]:
pivot_table['passed_incept_notfixated_forgone'].mean()

2.275

In [83]:
result2 = all_sub_df.groupby(['subject', 'condition', 'viewing_status']).size().reset_index(name='count')
result2

,subject,condition,viewing_status,count
0,sub001,Free_Viewing,passed_first_fixated,17
1,sub001,Free_Viewing,passed_first_notfixated_forgone,7
2,sub001,Free_Viewing,passed_incept_fixated,7
3,sub002,Count_trash_bins,passed_first_fixated,3
4,sub002,Count_trash_bins,passed_first_notfixated_forgone,8
...,...,...,...,...
130,sub039,Free_Viewing,passed_incept_fixated,2
131,sub040,Count_trash_bins,passed_first_fixated,3
132,sub040,Count_trash_bins,passed_first_notfixated_forgone,9
133,sub040,Count_trash_bins,passed_incept_fixated,5


In [87]:
pivot_table = result2.pivot_table(index='subject', columns='viewing_status', values='count', fill_value=0)
pivot_table

condition          Count_trash_bins                                  \
viewing_status passed_first_fixated passed_first_notfixated_forgone   
subject                                                               
sub001                            0                               0   
sub002                            3                               8   
sub003                            0                               0   
sub004                            1                              14   
sub005                            0                               0   
sub006                            0                               0   
sub007                            0                               0   
sub008                            5                              15   
sub009                            0                               0   
sub010                            2                               5   
sub011                            0                               0   
sub012                            5                               6   
sub013                            0                               0   
sub014                            0                               2   
sub015                            3                               4   
sub016                            6                              15   
sub017                            0                               0   
sub018                            0                               5   
sub019                            0                               0   
sub020                            2                              10   
sub021                            0                               0   
sub022                            4                              10   
sub023                            0                               0   
sub024                            3                              11   
sub025                            0                               0   
sub026                            4                               8   
sub027                            0                               0   
sub028                            0                               4   
sub029                            0                               0   
sub030                            3                              12   
sub031                            0                               0   
sub032                            6                              18   
sub033                            0                               0   
sub034                            2                              12   
sub035                            0                               0   
sub036                            6                               8   
sub037                            0                               0   
sub038                            3                              10   
sub039                            0                               0   
sub040                            3                               9   

condition                                                              \
viewing_status passed_incept_fixated passed_incept_notfixated_forgone   
subject                                                                 
sub001                             0                                0   
sub002                             5                                3   
sub003                             0                                0   
sub004                             0                                5   
sub005                             0                                0   
sub006                             0                                0   
sub007                             0                                0   
sub008                             2                                2   
sub009                             0                                0   
sub010                             2                                5 

In [ ]:
current_subject = sub_list[0]
print(current_subject)

#### Set up the file names to be read for this subject
position_names_file        = '../data/01_gaze_data/' + current_subject + '_billboard_list_position_name_assignment.csv'
passing_viewing_list_file  = '../data/01_gaze_data/' + current_subject + '_lists_1.txt'
recall_file                = '../data/03_interview_qualtrics_data/recall_sheet.csv'
recognition_file           = '../data/03_interview_qualtrics_data/recognition_sheet.csv'

#######################################################################################

#### Reading the position_name file
#### this gives us the "raw names" of which billboards were for this ppt at which position, which will be used to relabel the output files.
df_position_names  = pd.read_csv(position_names_file)
df_position_names['billboard_name_at_position_cleaned'] = df_position_names['billboard_image_at_position'].apply(lambda x: x.replace("resources/billboards/", "").replace(".bmp", ""))
df_dropped = df_position_names.drop(['index', 'position_index', 'billboard_image_at_position'], axis=1)
mapping_dict = pd.Series(df_dropped['billboard_name_at_position_cleaned'].values, index=df_dropped['billboard_named_position']).to_dict()

#######################################################################################

#### Reading the file with the remembered billboards
#### This gives us the behavioral information, i.e. which bbs were looked at, not looked at, incepted, etc.
with open(passing_viewing_list_file, 'r', encoding='utf-8') as file:
    data = file.read()
parsed_data = parse_data(data)
df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in parsed_data.items()])) # Convert to a DataFrame (optional)

passed_fixated_first_list = df['passed_fixated_first_list'].dropna()
passed_fixated_first_list = list(passed_fixated_first_list.values)
passed_fixated_first_list = [mapping_dict.get(item, item) for item in passed_fixated_first_list]

passed_fixated_incepted_list = df['passed_fixated_incepted_list'].dropna()
passed_fixated_incepted_list = list(passed_fixated_incepted_list.values)
passed_fixated_incepted_list = [mapping_dict.get(item, item) for item in passed_fixated_incepted_list]

repeated_forgone_list = df['repeated_forgone_list'].dropna()
repeated_forgone_list = list(repeated_forgone_list.values)
repeated_forgone_list = [mapping_dict.get(item, item) for item in repeated_forgone_list]

forgone_list = df['forgone_list'].dropna()
forgone_list = list(forgone_list.values)
forgone_list = [mapping_dict.get(item, item) for item in forgone_list]

passed_list = df['passed_list'].dropna()
passed_list = list(passed_list.values)
passed_list = [mapping_dict.get(item, item) for item in passed_list]

glanced_list = df['glanced_list'].dropna()
glanced_list = list(glanced_list.values)
glanced_list = [mapping_dict.get(item, item) for item in glanced_list]

#######################################################################################

#### Reading the recall data file
#### this gives us this participant's memory
recall_df = pd.read_csv(recall_file)
this_sub_df = recall_df[recall_df['subject'] == current_subject]
this_sub_recalled = this_sub_df.columns[this_sub_df.iloc[0] == 1].tolist()
#print(this_sub_recalled)

#######################################################################################

#### Combining the viewing lists with the recalls and compute memory results for each cell
# passed_fixated_first_list: determine which items from the recall_list where in the passed_fixated_first_list
common_elements = [item for item in passed_fixated_first_list if item in this_sub_recalled]
recall_passed_fixated_first = len(common_elements)/len(passed_fixated_first_list) if len(passed_fixated_first_list) != 0 else 0

# passed_fixated_incepted_list: determine which items from the recall_list where in the passed_fixated_incepted_list
common_elements = [item for item in passed_fixated_incepted_list if item in this_sub_recalled]
recall_fixated_incepted = len(common_elements)/len(passed_fixated_incepted_list) if len(passed_fixated_incepted_list) != 0 else 0

# repeated_forgone_list: determine which items from the recall_list where in the repeated_forgone_list
common_elements = [item for item in repeated_forgone_list if item in this_sub_recalled] 
recall_repeated_forgone = len(common_elements)/len(repeated_forgone_list) if len(repeated_forgone_list) != 0 else 0

# forgone_list: determine which items from the recall_list where in the forgone_list
common_elements = [item for item in forgone_list if item in this_sub_recalled] 
recall_forgone = len(common_elements)/len(forgone_list) if len(forgone_list) != 0 else 0

# glanced_list: determine which items from the recall_list where in the glanced_list
common_elements = [item for item in glanced_list if item in this_sub_recalled] 
recall_glanced = len(common_elements)/len(glanced_list) if len(glanced_list) != 0 else 0

#######################################################################################

#### Reading the Recognition Data
recognition_df = pd.read_csv(recognition_file)
this_sub_df = recognition_df[recognition_df['subject'] == current_subject]
this_sub_recognized = this_sub_df.columns[this_sub_df.iloc[0] == 1].tolist()
#print(this_sub_recognized)

#######################################################################################

# passed_fixated_first_list: determine which items from the recall_list where in the passed_fixated_first_list
common_elements = [item for item in passed_fixated_first_list if item in this_sub_recognized]
recognition_passed_fixated_first = len(common_elements)/len(passed_fixated_first_list) if len(passed_fixated_first_list) != 0 else 0

# passed_fixated_incepted_list: determine which items from the recall_list where in the passed_fixated_incepted_list
common_elements = [item for item in passed_fixated_incepted_list if item in this_sub_recognized]
recognition_fixated_incepted = len(common_elements)/len(passed_fixated_incepted_list) if len(passed_fixated_incepted_list) != 0 else 0

# repeated_forgone_list: determine which items from the recall_list where in the repeated_forgone_list
common_elements = [item for item in repeated_forgone_list if item in this_sub_recognized] 
recognition_repeated_forgone = len(common_elements)/len(repeated_forgone_list) if len(repeated_forgone_list) != 0 else 0

# forgone_list: determine which items from the recall_list where in the forgone_list
common_elements = [item for item in forgone_list if item in this_sub_recognized] 
recognition_forgone = len(common_elements)/len(forgone_list) if len(forgone_list) != 0 else 0

# forgone_list: determine which items from the recall_list where in the forgone_list
common_elements = [item for item in glanced_list if item in this_sub_recognized] 
recognition_glanced = len(common_elements)/len(glanced_list) if len(glanced_list) != 0 else 0

